## IO多路复用




### 客户端

In [ ]:
# 客户端
#tcp_client.py

from socket import *

#创建套接字
sockfd = socket()

#发起连接请求
server_addr = ('127.0.0.1',8888)
sockfd.connect(server_addr)

#消息收发
while True:
    data = input(">>")
    if not data:
        break
    sockfd.send(data.encode())
    data = sockfd.recv(1024)
    print("From server:",data.decode())

sockfd.close()









### select(服务端)

In [ ]:
from select import select 
from socket import *

#准备要关注的IO
s = socket()
s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
s.bind(('0.0.0.0',8888))
s.listen(3)

#添加关注列表
rlist = [s]
wlist = []
xlist = [s]

while True:
    #监控IO的发生
    rs,ws,xs = select(rlist,wlist,xlist)

    #遍历三个列表确定哪个IO发生
    for r in rs:
        #如果遍历到s说明s就绪则有客户端发起连接
        if r is s:
            c,addr = r.accept()
            print("Connect from",addr)
            rlist.append(c)
        #客户端连接套接子就绪,则接收消息
        else:
            data = r.recv(1024)
            if not data:
                #客户端退出从关注列表移出
                rlist.remove(r)
                r.close()
                continue 
            print("Receive from %s:%s"%(\
            r.getpeername(),data.decode()))
            # r.send(b'Receive')
            wlist.append(r)
    
    for w in ws:
        w.send(b"Receive your message") 
        wlist.remove(w)
    
    for x in xs:
        x.close()
        raise 




## POLL

In [ ]:

from socket import *
from select import *

#创建套接子
s = socket()
s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
s.bind(('0.0.0.0',8888))
s.listen(3)

#创建poll对象
p = poll() 

#建立查找字典
fdmap = {s.fileno():s}

#注册关注IO
p.register(s,POLLIN|POLLERR)

while True:
    events = p.poll()  #监控IO
    for fd,event in events:
        if fd == s.fileno():
            c,addr = fdmap[fd].accept()
            print("Connect from",addr)
            #添加新的关注事件
            p.register(c,POLLIN|POLLHUP)
            fdmap[c.fileno()] = c
        #通过按位与判断是否是某个事件就绪
        elif event & POLLIN:
            data = fdmap[fd].recv(1024)
            if not data:
                #客户端退出则取消关注,从字典删除
                p.unregister(fd)
                fdmap[fd].close()
                del fdmap[fd]
            else:
                print("Receive:",data.decode())
                fdmap[fd].send(b"Receive")




### EPOLL

In [ ]:
from socket import *
from select import *

#创建套接子
s = socket()
s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
s.bind(('0.0.0.0',8888))
s.listen(3)

#创建epoll对象
p = epoll() 

#建立查找字典
fdmap = {s.fileno():s}

#注册关注IO
p.register(s,EPOLLIN|EPOLLERR)

while True:
    events = p.poll()  #监控IO
    print("你有需要处理的IO哦...")
    for fd,event in events:
        if fd == s.fileno():
            c,addr = fdmap[fd].accept()
            print("Connect from",addr)
            #添加新的关注事件 设置边缘触发
            p.register(c,EPOLLIN|EPOLLHUP|EPOLLET)
            fdmap[c.fileno()] = c
        #通过按位与判断是否是某个事件就绪
        # elif event & EPOLLIN:
        #     data = fdmap[fd].recv(1024)
        #     if not data:
        #         #客户端退出则取消关注,从字典删除
        #         p.unregister(fd)
        #         fdmap[fd].close()
        #         del fdmap[fd]
        #     else:
        #         print("Receive:",data.decode())
        #         fdmap[fd].send(b"Receive")




### 本地socket

In [ ]:
# 接收
from socket import *
import os 

#如果文件已经存在则删除
if os.path.exists("./sock"):
    os.remove('./sock')

#创建本地套接子
sockfd = socket(AF_UNIX,SOCK_STREAM)

#绑定套接子文件
sockfd.bind("./sock")
#监听
sockfd.listen(3)

while True:
    c,addr = sockfd.accept()
    while True:
        data = c.recv(1024)
        if not data:
            break
        print(data.decode())
    c.close()
sockfd.close()


In [ ]:
# 发送
from socket import * 

sockfd = socket(AF_UNIX,SOCK_STREAM)

#两端需要使用相同的套接子文件
sockfd.connect('./sock') 

while True:
    msg = input(">>")
    if not msg:
        break 
    sockfd.send(msg.encode())

sockfd.close()

## Forkserver

In [ ]:
# -*- coding: UTF-8 -*-
from socket import * 
import os,sys 

#客户端处理函数
def client_handler(c):
    print("客户端:",c.getpeername())
    while True:
        data = c.recv(1024)
        if not data:
            break
        print(data.decode())
        c.send(b"Thank you!")
    c.close()

#创建套接子
HOST = '0.0.0.0'
PORT = 8888
ADDR = (HOST,PORT)

s = socket() #TCP套接字
s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
s.bind(ADDR)
s.listen(5)

#循环等待接收客户端连接请求
print("Listen to the port 8888....")
while True:
    try:
        c,addr = s.accept()
    except KeyboardInterrupt:
        sys.exit("退出服务器")
    except Exception as e:
        print("Error:",e)
        continue 
    
    #创建新的进程处理客户端请求
    pid = os.fork()

    if pid == 0:
        p = os.fork()
        if p == 0: #二级子进程
            s.close()
            client_handler(c) #处理具体请求
            sys.exit(0) #子进程处理完请求即退出
        else:
            os._exit(0)
    #父进程或者创建进程失败都继续等待下一个客户端连接
    else:
        c.close()
        os.wait()

